In [9]:
import pandas as pd
import numpy as np
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import GradientBoostingClassifier, BaggingClassifier, StackingClassifier, RandomForestClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
import tensorflow as tf
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
import pickle

In [ ]:
# N_SAMPLES = 20000

In [ ]:
df = pd.read_csv('./datasets/csgo_processed.csv', sep=';')

orig_X = df.drop(['bomb_planted'], axis=1).values
orig_y = df['bomb_planted'].values

oversampler = SMOTE()
X, y = oversampler.fit_resample(orig_X, orig_y)

ss = StandardScaler()
ss.fit(X)
with open("./models/scaler.pkl", "wb") as f:
    pickle.dump(ss, f)
X = ss.fit_transform(X)
categorical_y = tf.keras.utils.to_categorical(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
cat_X_train, cat_X_test, cat_y_train, cat_y_test = train_test_split(X, categorical_y, test_size=0.2, random_state=42, stratify=categorical_y)

In [12]:
gnb = GaussianNB()
cbc = CatBoostClassifier(learning_rate=0.2, n_estimators=100, max_depth=5)
bc = BaggingClassifier(n_estimators=100, max_features=0.5, max_samples=0.75)
sc = StackingClassifier(cv=5, final_estimator=LogisticRegression(), estimators=[('lr', LogisticRegression()), ('rf', RandomForestClassifier()), ('svc', SVC(probability=True))])
xgbc = XGBClassifier()
nn = tf.keras.models.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(2, activation='softmax')
])

In [13]:
gnb.fit(X_train, y_train)
print("GNB score:", gnb.score(X_test, y_test))
cbc.fit(X_train, y_train)
print("GBC score:", cbc.score(X_test, y_test))
bc.fit(X_train, y_train)
print("BC score:", bc.score(X_test, y_test))
sc.fit(X_train, y_train)
print("SC score:", sc.score(X_test, y_test))
xgbc.fit(X_train, y_train)
print("XGBC score:", xgbc.score(X_test, y_test))

GNB score: 0.904491993765056
0:	learn: 0.5392951	total: 5.36ms	remaining: 530ms
1:	learn: 0.4366116	total: 8.99ms	remaining: 440ms
2:	learn: 0.3632885	total: 12.8ms	remaining: 415ms
3:	learn: 0.3079943	total: 16.5ms	remaining: 395ms
4:	learn: 0.2643246	total: 21.1ms	remaining: 401ms
5:	learn: 0.2253354	total: 24.8ms	remaining: 389ms
6:	learn: 0.1994599	total: 28.4ms	remaining: 377ms
7:	learn: 0.1749377	total: 32ms	remaining: 368ms
8:	learn: 0.1562591	total: 36.4ms	remaining: 368ms
9:	learn: 0.1422198	total: 40.4ms	remaining: 363ms
10:	learn: 0.1298490	total: 44ms	remaining: 356ms
11:	learn: 0.1162558	total: 47.6ms	remaining: 349ms
12:	learn: 0.1051907	total: 52ms	remaining: 348ms
13:	learn: 0.0990224	total: 56ms	remaining: 344ms
14:	learn: 0.0952764	total: 59.4ms	remaining: 337ms
15:	learn: 0.0908024	total: 63.5ms	remaining: 333ms
16:	learn: 0.0866012	total: 67.8ms	remaining: 331ms
17:	learn: 0.0833079	total: 72.2ms	remaining: 329ms
18:	learn: 0.0805893	total: 76ms	remaining: 324ms
19:

In [14]:
nn.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
nn.fit(cat_X_train, cat_y_train, epochs=100, batch_size=32, validation_data=(cat_X_test, cat_y_test))
loss, accuracy = nn.evaluate(cat_X_test, cat_y_test, verbose=0)
print("NN test accuracy:", accuracy)

Epoch 1/100
883/883 [==============================] - 2s 2ms/step - loss: 0.1941 - accuracy: 0.9273 - val_loss: 0.1595 - val_accuracy: 0.9476
Epoch 2/100
883/883 [==============================] - 1s 2ms/step - loss: 0.1403 - accuracy: 0.9527 - val_loss: 0.1422 - val_accuracy: 0.9524
Epoch 3/100
883/883 [==============================] - 1s 2ms/step - loss: 0.1316 - accuracy: 0.9554 - val_loss: 0.1504 - val_accuracy: 0.9514
Epoch 4/100
883/883 [==============================] - 1s 2ms/step - loss: 0.1253 - accuracy: 0.9574 - val_loss: 0.1418 - val_accuracy: 0.9532
Epoch 5/100
883/883 [==============================] - 1s 1ms/step - loss: 0.1198 - accuracy: 0.9590 - val_loss: 0.1298 - val_accuracy: 0.9564
Epoch 6/100
883/883 [==============================] - 1s 2ms/step - loss: 0.1150 - accuracy: 0.9596 - val_loss: 0.1259 - val_accuracy: 0.9590
Epoch 7/100
883/883 [==============================] - 1s 1ms/step - loss: 0.1098 - accuracy: 0.9618 - val_loss: 0.1204 - val_accuracy: 0.9589

In [15]:
with open('./models/gnb.pkl', 'wb') as f:
    pickle.dump(gnb, f)
with open('./models/cbc.pkl', 'wb') as f:
    pickle.dump(cbc, f)
with open('./models/bc.pkl', 'wb') as f:
    pickle.dump(bc, f)
with open('./models/sc.pkl', 'wb') as f:
    pickle.dump(sc, f)
with open('./models/xgbc.pkl', 'wb') as f:
    pickle.dump(xgbc, f)
nn.save('./models/nn_model.h5')

c:\Users\csr10\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [16]:
with open('./models/gnb.pkl', 'rb') as f:
    gnb = pickle.load(f)
with open('./models/cbc.pkl', 'rb') as f:
    cbc = pickle.load(f)
with open('./models/bc.pkl', 'rb') as f:
    bc = pickle.load(f)
with open('./models/sc.pkl', 'rb') as f:
    sc = pickle.load(f)
with open('./models/xgbc.pkl', 'rb') as f:
    xgbc = pickle.load(f)
nn = tf.keras.models.load_model('./models/nn_model.h5')